# Obtener registros actualizados de incidentes

In [2]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
path = "/usr/bin/chromedriver"
service = Service(executable_path=path)
chrome_options = Options()
chrome_options.add_argument("--disable-javascript")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get("https://mapaemergencia.enel.com/mapa.html")

# Extrae las cookies de la sesión.
cookies_selenium = driver.get_cookies()
driver.quit()


# Cerrar Selenium
driver.quit()

# Convertir las cookies al formato requerido por requests
cookies = {cookie['name']: cookie['value'] for cookie in cookies_selenium}
# Configurar la solicitud GET
url_comunas = "https://mapaemergencia.enel.com/galeria/documento/me-capa-comunasAfectadas.txt"
url_alim_afectados = "https://mapaemergencia.enel.com/galeria/documento/me-capa-alimAfectados.txt"
url_trafos_afectados = "https://mapaemergencia.enel.com/galeria/documento/me-capa-trafosAfectados.txt"
url_capa_estado = "https://mapaemergencia.enel.com/galeria/documento/me-capa-estado.txt"
url_capa_avisos = "https://mapaemergencia.enel.com/galeria/documento/me-capa-avisos.txt"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.86 Safari/537.36",
    "Accept": "*/*",
    "Referer": "https://mapaemergencia.enel.com/mapa.html",
}

# Realizar la solicitud GET y obtener json con response*.text
response_comunas_afectadas = requests.get(url_comunas, headers=headers, cookies=cookies)
response_alim_afectados = requests.get(url_alim_afectados, headers=headers, cookies=cookies)
response_trafos_afectados = requests.get(url_trafos_afectados, headers=headers, cookies=cookies)
response_capa_estados = requests.get(url_capa_estado, headers=headers, cookies=cookies)
response_capa_avisos = requests.get(url_capa_avisos,headers=headers, cookies=cookies)


# Resumen de incidencias en RM

In [7]:
import json

# Cargar el JSON
data = json.loads(response_comunas_afectadas.text)

# Crear una lista para almacenar las filas del DataFrame
rows = []

# Extraer datos resumidos
for feature in data["features"]:
    props = feature["properties"]
    rows.append({
        "Comuna": props["COMUNA"],
        "Área": props["AREA"],
        "Perímetro": props["PERIMETER"],
        "Clientes Totales": props["CLIENTESTOTAL"],
        "Clientes Afectados": props["CLIENTESAFECTADOS"],
        "Porcentaje Afectados": props["PORCENTAJE"]
    })

# Crear el DataFrame
comunas_afectadas = pd.DataFrame(rows)

# Mostrar el DataFrame
comunas_afectadas


,Comuna,Área,Perímetro,Clientes Totales,Clientes Afectados,Porcentaje Afectados
0,CERRILLOS,1.674884e+07,21120.83490,29831,0,0.0
1,CERRO_NAVIA,1.117418e+07,16731.38365,34147,0,0.0
2,COLINA,9.649932e+08,161070.80801,54431,5,0.0
3,COLINA,9.649932e+08,161070.80801,,0,0
4,CONCHALI,1.103156e+07,14028.00043,36411,27,0.1
5,ESTACION_CENTRAL,1.427769e+07,18671.10646,75635,3,0.0
6,HUECHURABA,4.828945e+07,31478.30314,32932,61,0.2
7,INDEPENDENCIA,7.197278e+06,11519.47481,48457,2,0.0
8,LAMPA,4.648702e+08,102235.95779,44966,6,0.0
9,LAS_CONDES,1.035867e+08,48175.34293,152677,358,0.2


# Avisos a dataframe

In [6]:
import json
import pandas as pd
data = json.loads(response_capa_avisos.text)

# Extraer datos en formato tabular
rows = []
for feature in data["features"]:
    coordinates = feature["geometry"]["coordinates"]
    properties = feature["properties"]
    row = {
        "Latitud": coordinates[1],
        "Longitud": coordinates[0],
        **properties
    }
    rows.append(row)

df = pd.DataFrame(rows)

df

,Latitud,Longitud,TIPO,CODIGO,DIRECCION,FALLA,DESC_EVENTO,COD_EVENTO,FECHA_REPOSICION,FECHA_INI
0,-33.525963,-70.691515,AVISO,728108577,VENUS 02825,TEN Actualizado a 22/01/2025 1,prueba,DF202571260749,22-01-2025 13:12,21-01-2025 22:12
1,-33.497339,-70.746849,AVISO,728130328,TAURO 3375,TEN Actualizado a 22/01/2025 1,prueba,DF202571260788,22-01-2025 16:35,21-01-2025 23:35
2,-33.479228,-70.612649,AVISO,728385474,3 PONIENTE 2607,--fono apagado jmh,prueba,DF202571261095,22-01-2025 17:22,22-01-2025 12:22
3,-33.380447,-70.680567,AVISO,728356976,TUPUNGATO 5018,TEN Actualizado a 22/01/2025 1,prueba,DF202571260818,22-01-2025 16:06,22-01-2025 05:58
4,-33.380447,-70.680567,AVISO,728244138,TUPUNGATO 5018,TEN Actualizado a 22/01/2025 1,prueba,DF202571260818,22-01-2025 16:06,22-01-2025 05:58
...,...,...,...,...,...,...,...,...,...,...
185,-33.272147,-70.754436,AVISO,728289807,LOS MIRLOS 3 H/2 CHACRA,--,prueba,DF202571260982,22-01-2025 15:31,22-01-2025 10:31
186,-33.247376,-70.711137,AVISOC,727143419,CAM LO PINTO LOS JARDINES DE COLINA PARC 13,TEN Actualizado a 22/01/2025 1,prueba,DF202571259471,22-01-2025 15:46,20-01-2025 13:46
187,-33.367030,-70.845285,AVISO,727711512,HIJUELA SEGUNDAS TAHUILTACA S/N LOTE 4 MZ/ B,TEN Actualizado a 22/01/2025 1,prueba,DF202571260173,22-01-2025 16:42,21-01-2025 11:42
188,-33.292252,-70.669198,AVISOC,727317088,FUNDO CASAS DEL ALBA 3 0PARC 29,TEN Actualizado a 22/01/2025 1,prueba,DF202571259692,22-01-2025 16:21,20-01-2025 17:21
